In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import load_workbook

# Chrome driver setup
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Function to scrape fund data
def scrape_fund_data(fund_name, search_keyword):
    driver = set_chrome_driver()
    data = {}

    try:
        # Open the fund search page
        url = 'https://www.fundguide.net/Fund/SimpleSearch'
        driver.get(url)
        time.sleep(0.5)

        # Search for the fund
        search_box = driver.find_element(By.ID, 'search_keyword')
        search_box.send_keys(search_keyword)
        search_box.send_keys(Keys.RETURN)
        time.sleep(1)

        # Select the fund link
        fund_link = driver.find_element(By.LINK_TEXT, fund_name)
        fund_link.click()
        time.sleep(0.5)

        # Extract performance data
        button = driver.find_element(By.XPATH, '//button[@class="btn--tab__fund" and @value="1"]')
        button.click()
        time.sleep(0.5)
        
        period_data = {
            "펀드명": fund_name,
            "3개월": driver.find_element(By.XPATH, "//table[@id='tbl_periodRtn']//tr[1]//td[1]").text,
            "6개월": driver.find_element(By.XPATH, "//table[@id='tbl_periodRtn']//tr[1]//td[2]").text,
            "1년": driver.find_element(By.XPATH, "//table[@id='tbl_periodRtn']//tr[1]//td[3]").text,
            "3년": driver.find_element(By.XPATH, "//table[@id='tbl_periodRtn']//tr[1]//td[4]").text
        }
        data.update(period_data)

        # Extract portfolio data
        button = driver.find_element(By.XPATH, '//button[@class="btn--tab__fund" and @value="4"]')
        button.click()
        time.sleep(1)
        
        portfolio_data = {
            "1위": driver.find_element(By.XPATH, "//table[@id='tbl_stkTop']//tr[1]//td[2]").text,
            "2위": driver.find_element(By.XPATH, "//table[@id='tbl_stkTop']//tr[2]//td[2]").text,
            "3위": driver.find_element(By.XPATH, "//table[@id='tbl_stkTop']//tr[3]//td[2]").text,
            "4위": driver.find_element(By.XPATH, "//table[@id='tbl_stkTop']//tr[4]//td[2]").text,
            "5위": driver.find_element(By.XPATH, "//table[@id='tbl_stkTop']//tr[5]//td[2]").text
        }
        data.update(portfolio_data)

    except Exception as e:
        print(f"Error occurred for {fund_name}: {e}")
    
    finally:
        driver.quit()
        
    return data

In [2]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "마이다스액티브가치증권투자회사(주식)C-Pe2", "search_keyword": "마이다스액티브가치증권투자회사"},
    {"fund_name": "한화PLUS고배당주증권상장지수투자신탁(주식)", "search_keyword": "한화PLUS고배당주증권상장지수투자신탁"},
    {"fund_name": "브레인코스닥벤처증권투자신탁(주식혼합)종류AE", "search_keyword": "브레인코스닥벤처증권투자신탁"},
    {"fund_name": "KBRISE비메모리반도체액티브증권상장지수투자신탁(주식)", "search_keyword": "RISE비메모리반도체액티브증권상장지수투자신탁"},
    {"fund_name": "한국투자ACE삼성그룹주SW 증권상장지수투자신탁(주식)", "search_keyword": "ACE삼성그룹주SW"},
    {"fund_name": "현대강소기업증권자투자신탁 1[주식]종류C-P2e", "search_keyword": "현대강소기업증권자투자신탁"},
    {"fund_name": "한국밸류지속성장ESG증권자투자신탁(주식)C-Re", "search_keyword": "한국밸류지속성장ESG증권자투자신탁"},
    {"fund_name": "VIP한국형가치투자증권자투자신탁[주식]C-P2e클래스", "search_keyword": "VIP한국형가치투자증권자투자신탁[주식]"},
    {"fund_name": "삼성KODEX200증권상장지수투자신탁[주식]", "search_keyword": "삼성KODEX200증권"}
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_dom_stock_act = pd.DataFrame(all_fund_data)
print(df_dom_stock_act)

                              펀드명    3개월     6개월      1년      3년        1위  \
0        마이다스액티브가치증권투자회사(주식)C-Pe2  -2.89  -13.29   -4.09    3.17      삼성전자   
1        한화PLUS고배당주증권상장지수투자신탁(주식)  -2.87   -0.93   33.23   40.49      KT&G   
2        브레인코스닥벤처증권투자신탁(주식혼합)종류AE  -1.71   -7.18   19.04   64.76  지아이이노베이션   
3  KBRISE비메모리반도체액티브증권상장지수투자신탁(주식)   5.83  -15.87  -10.78    2.65      삼성전자   
4   한국투자ACE삼성그룹주SW 증권상장지수투자신탁(주식)  -6.77  -13.99   -3.25  -12.33      삼성전자   
5      현대강소기업증권자투자신탁 1[주식]종류C-P2e  -2.60  -25.20  -17.85    1.34      삼성전자   
6      한국밸류지속성장ESG증권자투자신탁(주식)C-Re   1.21   -6.87    9.95   14.86      삼성전자   
7   VIP한국형가치투자증권자투자신탁[주식]C-P2e클래스   2.83    0.38   13.77           메리츠금융지주   
8        삼성KODEX200증권상장지수투자신탁[주식]  -2.96  -11.63    1.83   -5.60      삼성전자   

         2위      3위     4위       5위  
0     DL이앤씨   S-Oil   제일기획      현대차  
1    우리금융지주   SK텔레콤   기업은행  BNK금융지주  
2  레인보우로보틱스      엠로    에스엠    더존비즈온  
3      리노공업    HPSP  한미반도체    DB하이텍  
4  삼성바이오로직스   삼성SDI   삼성물산   

In [3]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "삼성KODEX미국반도체MV증권상장지수투자신탁[주식]", "search_keyword": "삼성KODEX미국반도체MV증권상장지수투자신탁[주식]"},
    {"fund_name": "미래에셋TIGER미국나스닥100증권상장지수투자신탁(주식)", "search_keyword": "미래에셋TIGER미국나스닥100증권"},
    {"fund_name": "유리필라델피아반도체인덱스증권자투자신탁UH[주식]ClassC-P1e", "search_keyword": "유리필라델피아반도체인덱스증권자투자신탁UH[주식]"},
    {"fund_name": "삼성KODEX미국S&P500에너지증권상장지수투자신탁[주식-파생형](합성)", "search_keyword": "에너지증권상장지수투자신탁"},
    {"fund_name": "미래에셋인도중소형포커스증권자투자신탁 1(주식)종류C-P2e", "search_keyword": "미래에셋인도중소형포커스증권"},
    {"fund_name": "삼성인도중소형FOCUS증권자투자신탁UH[주식]Cpe(퇴직연금)", "search_keyword": "삼성인도중소형FOCUS증권자투자신탁UH"},
    {"fund_name": "삼성KODEX미국S&P500TotalReturn증권상장지수투자신탁[주식]", "search_keyword": "500TotalReturn증권상장지수투자신탁"},
    {"fund_name": "한국투자ACE미국S&P500증권상장지수투자신탁(주식)", "search_keyword": "500증권상장지수투자신탁"}
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_oversea_stock_act = pd.DataFrame(all_fund_data)
print(df_oversea_stock_act)

Error occurred for 삼성KODEX미국S&P500에너지증권상장지수투자신탁[주식-파생형](합성): Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table[@id='tbl_stkTop']//tr[1]//td[2]"}
  (Session info: chrome=132.0.6834.111); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x010574A3+25091]
	(No symbol) [0x00FDDC04]
	(No symbol) [0x00EBB373]
	(No symbol) [0x00EFF4DC]
	(No symbol) [0x00EFF65B]
	(No symbol) [0x00F3D8E2]
	(No symbol) [0x00F21F54]
	(No symbol) [0x00F3B49E]
	(No symbol) [0x00F21CA6]
	(No symbol) [0x00EF31D5]
	(No symbol) [0x00EF435D]
	GetHandleVerifier [0x013507C3+3142947]
	GetHandleVerifier [0x01361A2B+3213195]
	GetHandleVerifier [0x0135C412+3191154]
	GetHandleVerifier [0x010F8720+685184]
	(No symbol) [0x00FE6E1D]
	(No symbol) [0x00FE3E18]
	(No symbol) [0x00FE3FB6]
	(No symbol) [0x00FD66F0]
	BaseThreadInitThunk [0x75FB5D49+25]
	RtlInitializeExcep

In [4]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "미래에셋TIGER미국달러단기채권액티브증권상장지수투자신탁(채권-파생형)", "search_keyword": "TIGER미국달러단기채권액티브증권"},
    {"fund_name": "Plus신종개인용MMF2종류C-e", "search_keyword": "Plus신종개인용MMF2종류"},
    {"fund_name": "삼성신종MMF종류형D 2(C-Pe)", "search_keyword": "삼성신종MMF종류형"},
    {"fund_name": "KBRISEKIS국고채30년Enhanced증권상장지수투자신탁(채권)", "search_keyword": "KBRISEKIS국고채30년"},
    {"fund_name": "코레이트셀렉트단기채증권투자신탁[채권]_C-P2e", "search_keyword": "코레이트셀렉트단기채증권투자신탁"},
    {"fund_name": "한국투자크레딧포커스ESG증권자투자신탁 1(채권)(C-Re)", "search_keyword": "한국투자크레딧포커스ESG증권자투자신탁"},
    {"fund_name": "다올공모주하이일드증권투자신탁[채권혼합]종류CPe", "search_keyword": "다올공모주하이일드증권투자신탁"},
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_bond_act = pd.DataFrame(all_fund_data)
print(df_bond_act)

Error occurred for 미래에셋TIGER미국달러단기채권액티브증권상장지수투자신탁(채권-파생형): Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table[@id='tbl_stkTop']//tr[2]//td[2]"}
  (Session info: chrome=132.0.6834.111); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x010574A3+25091]
	(No symbol) [0x00FDDC04]
	(No symbol) [0x00EBB373]
	(No symbol) [0x00EFF4DC]
	(No symbol) [0x00EFF65B]
	(No symbol) [0x00F3D8E2]
	(No symbol) [0x00F21F54]
	(No symbol) [0x00F3B49E]
	(No symbol) [0x00F21CA6]
	(No symbol) [0x00EF31D5]
	(No symbol) [0x00EF435D]
	GetHandleVerifier [0x013507C3+3142947]
	GetHandleVerifier [0x01361A2B+3213195]
	GetHandleVerifier [0x0135C412+3191154]
	GetHandleVerifier [0x010F8720+685184]
	(No symbol) [0x00FE6E1D]
	(No symbol) [0x00FE3E18]
	(No symbol) [0x00FE3FB6]
	(No symbol) [0x00FD66F0]
	BaseThreadInitThunk [0x75FB5D49+25]
	RtlInitializeExcepti

In [5]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "한국밸류10년투자어린이증권투자신탁 1(주식)(C-E)", "search_keyword": "한국밸류10년투자어린이증권투자신탁"},
    {"fund_name": "한국투자ACE주주환원가치주액티브증권상장지수투자신탁(주식)", "search_keyword": "ACE 주주환원가치주 액티브"},
    {"fund_name": "미래에셋TIGERFn반도체TOP10증권상장지수투자신탁(주식)", "search_keyword": "TIGER Fn반도체TOP10"},
    {"fund_name": "삼성KODEXK-로봇액티브증권상장지수투자신탁[주식]", "search_keyword": "삼성KODEXK-로봇액티브증권상장지수투자신탁"},
    {"fund_name": "삼성KODEXK-신재생에너지액티브증권상장지수투자신탁[주식]", "search_keyword": "삼성KODEXK-신재생에너지액티브증권"},
    #{"fund_name": "한국투자ACE라이프자산주주가치액티브증권상장지수투자신탁(주식)", "search_keyword": "ACE 라이프자산주주가치액티브"},
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_dom_stock_ver = pd.DataFrame(all_fund_data)
print(df_dom_stock_ver)

                                 펀드명    3개월     6개월     1년     3년        1위  \
0      한국밸류10년투자어린이증권투자신탁 1(주식)(C-E)   6.46  -10.15  11.51  20.41      삼성전자   
1    한국투자ACE주주환원가치주액티브증권상장지수투자신탁(주식)   4.89   10.31  22.07           세아제강지주   
2  미래에셋TIGERFn반도체TOP10증권상장지수투자신탁(주식)   2.71  -17.19   0.94  14.20      삼성전자   
3       삼성KODEXK-로봇액티브증권상장지수투자신탁[주식]  12.51    0.11  -6.24            NAVER   
4   삼성KODEXK-신재생에너지액티브증권상장지수투자신탁[주식]   2.26   -2.76  34.01  61.85  HD현대일렉트릭   

         2위       3위        4위       5위  
0    SK하이닉스  HD현대중공업  HD한국조선해양     삼양식품  
1    더블유게임즈  영원무역홀딩스      SK가스      골프존  
2    SK하이닉스    한미반도체      리노공업     HPSP  
3  레인보우로보틱스       KT      LG전자  삼성에스디에스  
4   SK오션플랜트  두산에너빌리티     씨에스윈드    두산퓨얼셀  


In [6]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "삼성KODEX미국나스닥100TotalReturn증권상장지수투자신탁[주식]", "search_keyword": "삼성KODEX미국나스닥100TotalReturn"},
    {"fund_name": "미래에셋TIGER미국테크TOP10INDXX증권상장지수투자신탁(주식)", "search_keyword": "미래에셋TIGER미국테크TOP10INDXX"},
    {"fund_name": "한국투자ACE글로벌반도체TOP4PlusSOLACTIVE상장지수투자신탁(주식)", "search_keyword": "한국투자ACE글로벌반도체TOP4PlusSOLACTIVE상장지수투자신탁"},
    {"fund_name": "한국투자ACE글로벌브랜드TOP10블룸버그증권상장지수투자신탁(주식)", "search_keyword": "한국투자ACE글로벌브랜드TOP10블룸버그증권상장지수투자신탁"},
    {"fund_name": "한국투자ACE미국WideMoat동일가중증권상장지수투자신탁(주식)", "search_keyword": "한국투자ACE미국WideMoat"},
    {"fund_name": "미래에셋TIGER라틴증권상장지수투자신탁(주식)", "search_keyword": "미래에셋TIGER라틴증권상장지수투자신탁"},
    {"fund_name": "삼성KODEX미국S&P500금융증권상장지수투자신탁[주식]", "search_keyword": "500금융증권상장지수투자신탁"},
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_oversea_stock_ver = pd.DataFrame(all_fund_data)
print(df_oversea_stock_ver)

                                          펀드명    3개월    6개월     1년      3년  \
0    삼성KODEX미국나스닥100TotalReturn증권상장지수투자신탁[주식]  11.58  14.54  36.55   79.78   
1       미래에셋TIGER미국테크TOP10INDXX증권상장지수투자신탁(주식)  14.85  19.00  58.34  111.71   
2  한국투자ACE글로벌반도체TOP4PlusSOLACTIVE상장지수투자신탁(주식)  10.70  10.15  59.76           
3        한국투자ACE글로벌브랜드TOP10블룸버그증권상장지수투자신탁(주식)  13.96  13.54  39.83           
4         한국투자ACE미국WideMoat동일가중증권상장지수투자신탁(주식)   4.10  12.49  23.88   58.93   
5                   미래에셋TIGER라틴증권상장지수투자신탁(주식)   0.19  -0.90  -1.74   40.25   
6             삼성KODEX미국S&P500금융증권상장지수투자신탁[주식]  11.03  21.33  43.41           

                             1위                                         2위  \
0                     APPLE Inc                                NVIDIA Corp   
1                     Apple Inc                                NVIDIA Corp   
2                   NVIDIA Corp  Taiwan Semiconductor Manufacturing Co Ltd   
3                     Tesla Inc                        JPMorgan

In [8]:
# List of funds to scrape
funds_to_scrape = [
    {"fund_name": "삼성KODEX미국10년국채선물증권상장지수투자신탁[채권-파생형]", "search_keyword": "KODEX미국10년국채선물"},
    {"fund_name": "키움KOSEF물가채KIS증권상장지수투자신탁[채권]", "search_keyword": "키움KOSEF물가채KIS증권상장지수투자신탁"},
    {"fund_name": "삼성KODEXESG종합채권(A-이상)액티브증권상장지수투자신탁[채권]", "search_keyword": "삼성KODEXESG종합채권"}
]

# Collect all fund data
all_fund_data = [scrape_fund_data(fund["fund_name"], fund["search_keyword"]) for fund in funds_to_scrape]

# Create DataFrame from collected data
df_bond_ver = pd.DataFrame(all_fund_data)
print(df_bond_ver)

Error occurred for 삼성KODEX미국10년국채선물증권상장지수투자신탁[채권-파생형]: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table[@id='tbl_stkTop']//tr[1]//td[2]"}
  (Session info: chrome=132.0.6834.111); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x010574A3+25091]
	(No symbol) [0x00FDDC04]
	(No symbol) [0x00EBB373]
	(No symbol) [0x00EFF4DC]
	(No symbol) [0x00EFF65B]
	(No symbol) [0x00F3D8E2]
	(No symbol) [0x00F21F54]
	(No symbol) [0x00F3B49E]
	(No symbol) [0x00F21CA6]
	(No symbol) [0x00EF31D5]
	(No symbol) [0x00EF435D]
	GetHandleVerifier [0x013507C3+3142947]
	GetHandleVerifier [0x01361A2B+3213195]
	GetHandleVerifier [0x0135C412+3191154]
	GetHandleVerifier [0x010F8720+685184]
	(No symbol) [0x00FE6E1D]
	(No symbol) [0x00FE3E18]
	(No symbol) [0x00FE3FB6]
	(No symbol) [0x00FD66F0]
	BaseThreadInitThunk [0x75FB5D49+25]
	RtlInitializeExceptionCh

In [9]:
# EXCEL 파일 경로 설정 (기존 파일에 쓰기)
file_path = 'C:/세부자산현황/세부자산_20250131_매매.xlsx'
#'E:/자산관리/자산/세부자산_20241231_매매.xlsx'

# 각 값에 대해 예외 처리하여 숫자로 변환
def safe_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None  # 변환할 수 없는 경우 None 반환 (필요에 따라 0.0 등으로 변경 가능)

In [17]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4, value5, value6, value7, value8, value9) in enumerate(
        zip(df_dom_stock_act['3개월'], 
            df_dom_stock_act['6개월'], 
            df_dom_stock_act['1년'], 
            df_dom_stock_act['3년'], 
            df_dom_stock_act['1위'], 
            df_dom_stock_act['2위'], 
            df_dom_stock_act['3위'], 
            df_dom_stock_act['4위'], 
            df_dom_stock_act['5위']
            ), start=2):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/U
    if not pd.isna(value5):
        ws[f'AT{idx}'].value = value5
    if not pd.isna(value6):        
        ws[f'AU{idx}'].value = value6
    if not pd.isna(value7):
        ws[f'AV{idx}'].value = value7
    if not pd.isna(value8):
        ws[f'AW{idx}'].value = value8
    if not pd.isna(value9):
        ws[f'AX{idx}'].value = value9

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.


In [18]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4, value5, value6, value7, value8, value9) in enumerate(
        zip(df_oversea_stock_act['3개월'], 
            df_oversea_stock_act['6개월'], 
            df_oversea_stock_act['1년'], 
            df_oversea_stock_act['3년'], 
            df_oversea_stock_act['1위'], 
            df_oversea_stock_act['2위'], 
            df_oversea_stock_act['3위'], 
            df_oversea_stock_act['4위'], 
            df_oversea_stock_act['5위']
            ), start=12):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/U
    if not pd.isna(value5):
        ws[f'AT{idx}'].value = value5
    if not pd.isna(value6):        
        ws[f'AU{idx}'].value = value6
    if not pd.isna(value7):
        ws[f'AV{idx}'].value = value7
    if not pd.isna(value8):
        ws[f'AW{idx}'].value = value8
    if not pd.isna(value9):
        ws[f'AX{idx}'].value = value9

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.


In [20]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4) in enumerate(
        zip(df_bond_act['3개월'], 
            df_bond_act['6개월'], 
            df_bond_act['1년'], 
            df_bond_act['3년'], 
            ), start=21):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/U

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.


In [22]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4, value5, value6, value7, value8, value9) in enumerate(
        zip(df_dom_stock_ver['3개월'], 
            df_dom_stock_ver['6개월'], 
            df_dom_stock_ver['1년'], 
            df_dom_stock_ver['3년'], 
            df_dom_stock_ver['1위'], 
            df_dom_stock_ver['2위'], 
            df_dom_stock_ver['3위'], 
            df_dom_stock_ver['4위'], 
            df_dom_stock_ver['5위']
            ), start=60):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/U
    if not pd.isna(value5):
        ws[f'AT{idx}'].value = value5
    if not pd.isna(value6):        
        ws[f'AU{idx}'].value = value6
    if not pd.isna(value7):
        ws[f'AV{idx}'].value = value7
    if not pd.isna(value8):
        ws[f'AW{idx}'].value = value8
    if not pd.isna(value9):
        ws[f'AX{idx}'].value = value9

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.


In [23]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4, value5, value6, value7, value8, value9) in enumerate(
        zip(df_oversea_stock_ver['3개월'], 
            df_oversea_stock_ver['6개월'], 
            df_oversea_stock_ver['1년'], 
            df_oversea_stock_ver['3년'], 
            df_oversea_stock_ver['1위'], 
            df_oversea_stock_ver['2위'], 
            df_oversea_stock_ver['3위'], 
            df_oversea_stock_ver['4위'], 
            df_oversea_stock_ver['5위']
            ), start=67):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/U
    if not pd.isna(value5):
        ws[f'AT{idx}'].value = value5
    if not pd.isna(value6):        
        ws[f'AU{idx}'].value = value6
    if not pd.isna(value7):
        ws[f'AV{idx}'].value = value7
    if not pd.isna(value8):
        ws[f'AW{idx}'].value = value8
    if not pd.isna(value9):
        ws[f'AX{idx}'].value = value9

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.


In [27]:
# EXCEL 파일 읽기
wb = load_workbook(file_path)
# 특정 시트 선택
ws = wb['관심펀드']

# 여러 컬럼 값을 동시에 세팅
for idx, (value1, value2, value3, value4) in enumerate(
        zip(df_bond_ver['3개월'], 
            df_bond_ver['6개월'], 
            df_bond_ver['1년'], 
            df_bond_ver['3년'], 
            ), start=77):

    ws[f'R{idx}'].value = safe_float(value1) # F/J/N/R
    ws[f'S{idx}'].value = safe_float(value2) # G/K/O/S
    ws[f'T{idx}'].value = safe_float(value3) # H/l/P/T
    ws[f'U{idx}'].value = safe_float(value4) # I/M/Q/UU

# 기존 파일에 저장
wb.save(file_path)

print(f"데이터가 성공적으로 {file_path}에 숫자형으로 저장되었습니다.")

데이터가 성공적으로 C:/세부자산현황/세부자산_20250131_매매.xlsx에 숫자형으로 저장되었습니다.
